<a href="https://colab.research.google.com/github/virf96/Basico/blob/main/REDUCCION_DIMENSIONALIDAD_y_selectionfeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
pd.set_option("display.max_columns",200)

# GENERAL

### Valor perdido



OBJETIVO - PREDECIR EL PRECIO DE ARTICULOS EN EL MERCADO A PARTIR DE SUS CARACTERISTICAS

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv("/content/drive/MyDrive/datasets/precio.csv")

In [4]:
df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [5]:
df["Item_Weight"]= df["Item_Weight"].mask(np.random.random(df["Item_Weight"].shape)<0.5)

df.to_csv("/home/carlita/Documents/Diplomado/Datos/archive (15)/precio.csv",index=False)

In [6]:
pd.DataFrame(df.isnull().sum()/len(df)*100)

,0
Item_Identifier,0.000000
Item_Weight,78.739880
Item_Fat_Content,0.000000
Item_Visibility,0.000000
Item_Type,0.000000
Item_MRP,0.000000
Outlet_Identifier,0.000000
Outlet_Establishment_Year,0.000000
Outlet_Size,28.276428
Outlet_Location_Type,0.000000


- Como puede ver en la tabla anterior,  hay demasiados valores perdidos (solo 2 variables los tienen). Podemos eliminar esas columnas. 

In [7]:
df.drop(columns=["Item_Weight","Outlet_Size"],inplace=True)

df['Item_Weight'].fillna(df['Item_Weight'].median(), inplace=True)
df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0], inplace=True)

In [8]:
pd.DataFrame(df.isnull().sum()/len(df)*100)

,0
Item_Identifier,0.0
Item_Fat_Content,0.0
Item_Visibility,0.0
Item_Type,0.0
Item_MRP,0.0
Outlet_Identifier,0.0
Outlet_Establishment_Year,0.0
Outlet_Location_Type,0.0
Outlet_Type,0.0
Item_Outlet_Sales,0.0


### Baja Varianza

In [9]:
#Calculamos la varianza de todas las variables

df.var()

Item_Visibility              2.662335e-03
Item_MRP                     3.878184e+03
Outlet_Establishment_Year    7.008637e+01
Item_Outlet_Sales            2.912141e+06
dtype: float64

- Como muestra el resultado anterior, la varianza de Item_Visibility es muy inferior en comparación con las otras variables. Podemos soltar esta columna con seguridad. Así es como aplicamos el filtro de baja varianza.

In [10]:
var=df.var()

In [11]:
varianza=pd.DataFrame(var).reset_index().rename(columns={"index":"col",0:"var"})

In [12]:
threshold=10

In [13]:
varianza[varianza["var"].astype(float)>threshold]

,col,var
1,Item_MRP,3.878184e+03
2,Outlet_Establishment_Year,7.008637e+01
3,Item_Outlet_Sales,2.912141e+06


In [14]:
varianza[varianza["var"].astype(float)<threshold]

,col,var
0,Item_Visibility,0.002662


In [15]:
df.drop(columns=["Item_Visibility"],inplace=True)

### Alta Correlación

OBJETIVO - PREDECIR EL SALARIO A PARTIR DE LAS CARACTERISTICAS DEL EMPLEADO

- Si la correlación entre un par de variables es mayor que 0.5-0.6, deberíamos considerar seriamente descartar una de esas variables.

In [16]:
df.corr()

,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Item_MRP,1.000000,0.005020,0.567574
Outlet_Establishment_Year,0.005020,1.000000,-0.049135
Item_Outlet_Sales,0.567574,-0.049135,1.000000


In [17]:
def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

In [18]:
fig = go.Figure(data=go.Heatmap(df_to_plotly(df.corr()),hoverongaps = False,colorscale='Inferno'))
fig.show()

In [19]:

fig = go.Figure(data=go.Heatmap(df_to_plotly(df.corr()),hoverongaps = False,colorscale='Viridis'))
fig.show()

In [21]:
data=pd.read_csv("/content/drive/MyDrive/datasets/Salary.csv")


In [22]:
data.corr()

,YearsExperience,Gender,Education_level,Age,Salary
YearsExperience,1.000000,-0.043824,-0.251161,0.726544,0.982427
Gender,-0.043824,1.000000,0.077152,0.013310,-0.046927
Education_level,-0.251161,0.077152,1.000000,-0.014377,-0.207260
Age,0.726544,0.013310,-0.014377,1.000000,0.731026
Salary,0.982427,-0.046927,-0.207260,0.731026,1.000000


### Correlación (con el objetivo)

In [23]:
data.corr()

,YearsExperience,Gender,Education_level,Age,Salary
YearsExperience,1.000000,-0.043824,-0.251161,0.726544,0.982427
Gender,-0.043824,1.000000,0.077152,0.013310,-0.046927
Education_level,-0.251161,0.077152,1.000000,-0.014377,-0.207260
Age,0.726544,0.013310,-0.014377,1.000000,0.731026
Salary,0.982427,-0.046927,-0.207260,0.731026,1.000000


### Multicolinealidad

OBJETIVO - PREDECIR EL SALARIO A PARTIR DE LAS CARACTERISTICAS DEL EMPLEADO

El valor de R ^ 2 se determina para averiguar qué tan bien se describe una variable independiente por las otras variables independientes. Un valor alto de R ^ 2 significa que la variable está altamente correlacionada con las otras variables. Esto es capturado por el VIF que se indica a continuación:


Entonces, cuanto más cerca esté el valor de R ^ 2 a 1, mayor será el valor de VIF y mayor será la multicolinealidad con la variable independiente particular.

**Factor de inflación de la varianza (VIF)**

- VIF comienza en 1 y no tiene límite superior
- VIF = 1, sin correlación entre la variable independiente y las otras variables
- VIF superior a 5 o 10 indica una alta multicolinealidad entre esta variable independiente y las demás

In [24]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [25]:
df=pd.read_csv("/content/drive/MyDrive/datasets/Salary.csv")

In [26]:
df=df[['YearsExperience', 'Gender', 'Education_level', 'Age', 'Salary']]

In [27]:
X = df.iloc[:,:-1]

In [30]:
X.head()

,YearsExperience,Gender,Education_level,Age
0,1.1,1,1,23
1,1.3,1,1,28
2,1.5,1,1,25
3,2.0,0,1,26
4,2.2,0,1,22


In [28]:
[variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

[7.681903224550903, 2.488769440966616, 3.1444437643289143, 14.27250831946706]

In [31]:
variance_inflation_factor(X.values, 1)

2.488769440966616

In [32]:
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [33]:
calc_vif(X)

,variables,VIF
0,YearsExperience,7.681903
1,Gender,2.488769
2,Education_level,3.144444
3,Age,14.272508


**Podemos ver aquí que la 'Edad' y los 'Años de servicio' tienen un valor VIF alto, lo que significa que pueden predecirse mediante otras variables independientes en el conjunto de datos.**

# CONJUNTO ENTRENAMIENTO (Requerido para Aprendizaje Superviado)

### PCA

**PCA es fundamentalmente un algoritmo de reducción de dimensionalidad, pero también puede ser útil como herramienta para visualización, filtrado de ruido, extracción e ingeniería de características**

OBJETIVO - PREDECIR EL TIPO DE ESPECIE QUE ES A TRAVÉS DE SUS CARACTERISTICAS

In [34]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [35]:
df = px.data.iris()
df.drop(columns=["species_id"],inplace=True)
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y=df["species"]

In [36]:
df["species"].value_counts()

setosa        50
virginica     50
versicolor    50
Name: species, dtype: int64

In [37]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [38]:
df.corr()

,sepal_length,sepal_width,petal_length,petal_width
sepal_length,1.000000,-0.109369,0.871754,0.817954
sepal_width,-0.109369,1.000000,-0.420516,-0.356544
petal_length,0.871754,-0.420516,1.000000,0.962757
petal_width,0.817954,-0.356544,0.962757,1.000000


In [39]:
fig = go.Figure(data=go.Heatmap(df_to_plotly(df.corr()),hoverongaps = False,colorscale='Viridis'))
fig.show()

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [41]:
X_train.shape

(120, 4)

In [42]:
scaler = StandardScaler()
scaler.fit(X_train)
X_sc=scaler.transform(X_train)

In [43]:
pca = PCA(n_components=3)
pca.fit(X_sc)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [44]:
X_pca=pd.DataFrame(pca.transform(X_sc))

In [45]:
#La cantidad de variación explicada por cada uno de los componentes seleccionados.
pca.explained_variance_

array([2.91333895, 0.96705055, 0.13464674])

In [46]:
#Porcentaje de varianza explicada por cada uno de los componentes seleccionados.
pca.explained_variance_ratio_

array([0.72226528, 0.23974795, 0.03338117])

In [47]:
sum(pca.explained_variance_ratio_[:-1])

0.9620132300547347

In [48]:
pca = PCA(n_components=2)
dimention_2=pca.fit_transform(X_sc)
fig = px.scatter(dimention_2, x=0, y=1,color=y_train,labels={'0': 'Componente Principal 1', '1': 'Componente Principal 2'})
fig.show()

In [49]:
pca = PCA(n_components=3)
dimention_3=pca.fit_transform(X_sc)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    dimention_3, x=0, y=1, z=2, color=y_train,
    title=f'Total de Varianza Explicada: {total_var:.2f}%',
    labels={'0': 'CP 1', '1': 'CP 2', '2': 'CP 3'}
)
fig.show()

### Arboles

OBJETIVO - PREDECIR EL PRECIO DE ARTICULOS EN EL MERCADO A PARTIR DE SUS CARACTERISTICAS

https://gdcoder.com/decision-tree-regressor-explained-in-depth/#:~:text=Decision%20trees%20regression%20normally%20use,the%20data%20into%20two%20subset.&text=The%20tree%20chooses%20the%20value%20with%20results%20in%20smallest%20MSE%20value.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
#XGBOOST tambien genere importancia de variables

In [50]:
df=pd.read_csv("/content/drive/MyDrive/datasets/data_arboles.csv")

In [51]:
X=df[[x for x in df if x!='Item_Outlet_Sales']]
y=df["Item_Outlet_Sales"]

In [52]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138000
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.422800
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.270000
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.380000
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.705200
...,...,...,...,...,...,...,...,...,...,...,...,...
14199,FDB58,10.50,Regular,0.013496,Snack Foods,141.3154,OUT046,1997,Small,Tier 1,Supermarket Type1,2181.288914
14200,FDD47,7.60,Regular,0.142991,Starchy Foods,169.1448,OUT018,2009,Medium,Tier 3,Supermarket Type2,2181.288914
14201,NCO17,10.00,Low Fat,0.073529,Health and Hygiene,118.7440,OUT045,2002,NaN,Tier 2,Supermarket Type1,2181.288914
14202,FDJ26,15.30,Regular,0.000000,Canned,214.6218,OUT017,2007,NaN,Tier 2,Supermarket Type1,2181.288914


In [53]:
X=X.drop(['Item_Identifier', 'Outlet_Identifier'], axis=1)

In [54]:
X=X[["Item_Weight","Item_Visibility","Item_MRP"]+["Item_Fat_Content","Item_Type","Outlet_Size","Outlet_Location_Type","Outlet_Type"]]

In [55]:
X.isnull().sum()

Item_Weight             2439
Item_Visibility            0
Item_MRP                   0
Item_Fat_Content           0
Item_Type                  0
Outlet_Size             4016
Outlet_Location_Type       0
Outlet_Type                0
dtype: int64

In [56]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [57]:
X_train.columns

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Fat_Content',
       'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [58]:
X_train['Item_Weight'].fillna(X_train['Item_Weight'].median(), inplace=True)
X_train['Outlet_Size'].fillna(X_train['Outlet_Size'].mode()[0], inplace=True)

X_test['Item_Weight'].fillna(X_train['Item_Weight'].median(), inplace=True)
X_test['Outlet_Size'].fillna(X_train['Outlet_Size'].mode()[0], inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [59]:
X_train["Item_Fat_Content"].value_counts()

Low Fat    6340
Regular    3662
LF          378
reg         145
low fat     128
Name: Item_Fat_Content, dtype: int64

In [60]:
fat_cont=pd.get_dummies(X_train[["Item_Fat_Content"]]).iloc[:,:-1]
item_type=pd.get_dummies(X_train[["Item_Type"]]).iloc[:,:-1]
out_size=pd.get_dummies(X_train[["Outlet_Size"]]).iloc[:,:-1]
out_type_loc=pd.get_dummies(X_train[["Outlet_Location_Type"]]).iloc[:,:-1]
out_type=pd.get_dummies(X_train[["Outlet_Type"]]).iloc[:,:-1]

X_train=pd.concat([X_train[["Item_Weight","Item_Visibility","Item_MRP"]],fat_cont,item_type,out_size,out_type_loc,out_type],axis=1)

In [61]:
X_train.shape

(10653, 29)

In [65]:
X_train.head(5)

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Outlet_Size_High,Outlet_Size_Medium,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2
6939,17.350,0.014722,73.1038,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
1917,15.600,0.037764,124.9704,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0
599,5.000,0.044005,188.8530,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
5933,12.500,0.007180,47.4034,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
10282,8.235,0.082412,148.5076,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [62]:
list(X_train.isnull().sum().values)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [67]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=1, max_depth=10)

In [68]:
model.fit(X_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=10,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=1, splitter='best')

In [69]:
features = df.columns
#Los modelos de arbol tienen el atributo de feature_importances
importances = model.feature_importances_

In [70]:
importa=pd.DataFrame()

In [71]:
importa["var"]=X_train.columns
importa["importance"]=model.feature_importances_

In [72]:
importa=importa.sort_values(by="importance",ascending=False)

In [73]:
importa[:20]

,var,importance
2,Item_MRP,0.500362
26,Outlet_Type_Grocery Store,0.240860
1,Item_Visibility,0.067903
28,Outlet_Type_Supermarket Type2,0.066514
0,Item_Weight,0.038878
27,Outlet_Type_Supermarket Type1,0.026322
13,Item_Type_Fruits and Vegetables,0.009076
11,Item_Type_Dairy,0.006643
6,Item_Fat_Content_low fat,0.005022
7,Item_Type_Baking Goods,0.004066


### Select KBEST

OBJETIVO - PREDECIR EL PRECIO DE ARTICULOS EN EL MERCADO A PARTIR DE SUS CARACTERISTICASm

https://portal.uah.es/portal/page/portal/epd2_asignaturas/asig202218/informacion_academica/2-Modelos%20lineales.pdf

https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

Score function:
- For regression: f_regression, mutual_info_regression
- For classification: chi2, f_classif, mutual_info_classif

**f_regression**
- Pruebas de regresión lineal univariante.

- Modelo lineal para probar el efecto individual de cada uno de los muchos regresores. Esta es una función de puntuación que se utilizará en un procedimiento de selección de características, no un procedimiento de selección de características independiente.

- Esto se hace en 2 pasos:

- Se calcula la correlación entre cada regresor y el objetivo, es decir, ((X [:, i] - mean (X [:, i])) * (y - mean_y)) / (std (X [:, i] ) * estándar (y)).

- Se convierte en una puntuación F y luego en un valor p.



In [74]:
from sklearn.feature_selection import SelectKBest,f_regression

In [75]:
X_train

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Outlet_Size_High,Outlet_Size_Medium,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2
6939,17.350,0.014722,73.1038,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
1917,15.600,0.037764,124.9704,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0
599,5.000,0.044005,188.8530,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
5933,12.500,0.007180,47.4034,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
10282,8.235,0.082412,148.5076,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12071,12.650,0.042450,107.3938,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0
1794,7.300,0.086061,149.8076,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
10823,20.200,0.100068,186.8898,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0
441,12.850,0.022604,181.4318,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [76]:
kb = SelectKBest(f_regression, k=20)

In [77]:
X_new=kb.fit_transform(X_train, y_train)

In [78]:
X_new.shape

(10653, 20)

In [79]:
X_train.columns[kb.get_support()]

Index(['Item_Visibility', 'Item_MRP', 'Item_Fat_Content_LF',
       'Item_Fat_Content_Regular', 'Item_Type_Baking Goods',
       'Item_Type_Breakfast', 'Item_Type_Fruits and Vegetables',
       'Item_Type_Health and Hygiene', 'Item_Type_Household',
       'Item_Type_Others', 'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Outlet_Size_High', 'Outlet_Size_Medium',
       'Outlet_Location_Type_Tier 1', 'Outlet_Location_Type_Tier 2',
       'Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type1',
       'Outlet_Type_Supermarket Type2'],
      dtype='object')

In [80]:
pd.DataFrame(X_new)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.014722,73.1038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.037764,124.9704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.044005,188.8530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.007180,47.4034,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.082412,148.5076,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10648,0.042450,107.3938,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
10649,0.086061,149.8076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
10650,0.100068,186.8898,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
10651,0.022604,181.4318,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
